In [176]:
import numpy as np
import re

In [177]:
# constants
BOARD_SIZE = 9
SECTOR_SIZE = 3

### Numpy vectorized functions

In [ ]:
get_length = np.vectorize(len)

cell_contains = np.vectorize(lambda x, y : y in x)

## Helper functions

In [326]:
def remove_value(arr, pattern):
    numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
    for val in pattern:
        for cell in numpy_iterator:
            if str(cell) != pattern:
                cell[...] = str(cell).replace(val,'')

def get_row(arr, coord):
    return arr[coord[0],:]

def get_col(arr, coord):
    return arr[:,coord[1]]

def get_sector(arr, sector):
    row = sector // SECTOR_SIZE * SECTOR_SIZE
    col = SECTOR_SIZE * (sector % SECTOR_SIZE)
    return arr[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE]

def get_quad(arr, coord):
    x_home = coord[0] // SECTOR_SIZE * SECTOR_SIZE
    y_home = coord[1] // SECTOR_SIZE * SECTOR_SIZE
    return arr[x_home:x_home+3,y_home:y_home+3]

def update_neighbors(arr, coord, val):
    remove_value(get_row(arr, coord), val)
    remove_value(get_col(arr, coord), val)
    remove_value(get_quad(arr, coord), val)

def pretty_flat(board):
    width = np.max(get_length(board)) + 2
    line = '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+'
    for idx, cell in enumerate(board):
        if not idx % 27:
            print(line)
        if not idx % 3:
            print('|', end='')
        print(F'{cell:^{width}s}', end='')
        if not ((idx + 1) % 9):
            print('|')
    print(line)

# this will run recursively until there are no more singletons left
def clean_singletons(dest,source):
    clean_counter = 0
    coordinate_list = np.argwhere(get_length(source)==1)
    while coordinate_list.size:
        clean_counter += 1
        for idc in coordinate_list:
            singleton = source[idc[0], idc[1]]
            row, col = idc
            # update the destination with the singleton value
            dest[row,col] = singleton

            # empty this cell in the source as it has been solved
            source[row,col] = '' 

            # remove the singleton from every neighbor
            update_neighbors(source, (row,col), singleton)

        # recursive call
        coordinate_list = np.argwhere(get_length(source)==1)

    return clean_counter

def clean_disguised_singletons(arr):
    altered_coords = set({})
    for val in '123456789':
        # search every row
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(arr[idx,:],val)) == 1:
                col = np.argwhere(cell_contains(arr[idx,:],val))[0][0]
                arr[idx,col] = val
                altered_coords.add((idx,col))

        # search every column
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(arr[:,idx],val)) == 1:
                row = np.argwhere(cell_contains(arr[:,idx],val))[0][0]
                arr[row,idx] = val
                altered_coords.add((row,idx))

        # search every sector
        for idx in range(BOARD_SIZE):
            if np.count_nonzero(cell_contains(get_sector(arr,idx),val)) == 1:
                coords = np.argwhere(cell_contains(get_sector(arr,idx),val))
                row = coords[0][0] + SECTOR_SIZE * (idx // SECTOR_SIZE)
                col = coords[0][1] + SECTOR_SIZE * (idx % SECTOR_SIZE)
                arr[row,col] = val
                altered_coords.add((row,col))
    return len(altered_coords)

---

### Load the puzzle

In [365]:
possible = np.full((9,9),'123456789')
board = np.full((9,9),'')

# hard
puzzle = re.sub('\W','0','2.......6.5..8..1...4...9...7.3.1......82.......7.5.3...9...4...8..1..5.6.......2')
# easy
puzzle = re.sub('\W','0','5..98.67.6......31.2.613.4..968.21.7..8..5.9.7.319....962.7..1.1.5...76..7.5..9..')
# medium
puzzle = re.sub('\W','0','.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52')
# hard
puzzle = re.sub('\W','0','..791.5....1.....3..9.4...2.4...83.....3.1....6..5...8.2..9...5...........4.8..7.')


for idx, value in enumerate(puzzle):
    if value != '0':
        row = idx // BOARD_SIZE
        col = idx - row * BOARD_SIZE
        possible[row,col] = value

solved = len(np.argwhere(get_length(np.array(list(re.sub('0','',puzzle))))==1))
print(F"Puzzle starts with {solved} solved locations.")

pretty_flat(board.flatten())
pretty_flat(possible.flatten())

Puzzle starts with 23 solved locations.
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
+---------------------------------+---------------------------------+---------------------------------+
| 123456789  123456789      7     |     9          1      123456789 |     5      123456789  123456789 |
| 123456789  123456789      1     | 123456789  123456789  123456789 | 123456789  123456789      3     |
| 123456789  123456789      9     | 123456789      4      123456789 | 123456789  123456789      2     |
+---------------------------------+---------------------------------+---------------------------------+
| 123456789      4      123456789 | 123456789  123456789      8     |     3      123456789  123456789 |
| 123456789  123456789  123456789 |  

---

### Step 1: Handle singletons

Look at every cell that has only one **possible** value. This is the only step that makes modifications to actual board. Therefore, this will be the first and last steps of solving any puzzle.

This cell calls a function that runs repeatedly until it cannot simplify the board any more.

In [383]:
response = clean_singletons(board,possible)
print(F"Performed {response} singleton sweeps.")

locations = len(np.argwhere(get_length(board)==1))
if locations == BOARD_SIZE**2:
    print("Board has been solved!!!")
    pretty_flat(board.flatten())
else:
    print(F"Puzzle currently has {locations} solved locations.")
    pretty_flat(board.flatten())
    pretty_flat(possible.flatten())


Performed 0 singleton sweeps.
Board has been solved!!!
+---------+---------+---------+
| 2  8  7 | 9  1  3 | 5  6  4 |
| 4  5  1 | 2  7  6 | 8  9  3 |
| 6  3  9 | 8  4  5 | 7  1  2 |
+---------+---------+---------+
| 1  4  2 | 7  6  8 | 3  5  9 |
| 5  9  8 | 3  2  1 | 6  4  7 |
| 7  6  3 | 4  5  9 | 1  2  8 |
+---------+---------+---------+
| 8  2  6 | 1  9  7 | 4  3  5 |
| 9  7  5 | 6  3  4 | 2  8  1 |
| 3  1  4 | 5  8  2 | 9  7  6 |
+---------+---------+---------+


---

### Step 2: Handle disguised singletons

In [382]:
response = clean_disguised_singletons(possible)
print(F"Located {response} disguised singletons.")
pretty_flat(possible.flatten())
pretty_flat(board.flatten())
clean_singletons(board,possible)

Located 0 disguised singletons.
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
|      |      |      |
|      |      |      |
|      |      |      |
+------+------+------+
+---------+---------+---------+
| 2  8  7 | 9  1  3 | 5  6  4 |
| 4  5  1 | 2  7  6 | 8  9  3 |
| 6  3  9 | 8  4  5 | 7  1  2 |
+---------+---------+---------+
| 1  4  2 | 7  6  8 | 3  5  9 |
| 5  9  8 | 3  2  1 | 6  4  7 |
| 7  6  3 | 4  5  9 | 1  2  8 |
+---------+---------+---------+
| 8  2  6 | 1  9  7 | 4  3  5 |
| 9  7  5 | 6  3  4 | 2  8  1 |
| 3  1  4 | 5  8  2 | 9  7  6 |
+---------+---------+---------+


0

---

### Step 3: Handle matching patterns

In [372]:
def get_common_patterns(arr):
    return_list = []
    pattern_dict = dict()
    numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
    for cell in numpy_iterator:
        pattern_dict[str(cell)] = pattern_dict.get(str(cell),0) + 1
    for k,v in pattern_dict.items():
        if len(k) == v:
            return_list.append(k)
    return return_list

def remove_pattern_from_group(arr, patterns):
    for patt in patterns:
        numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
        for cell in numpy_iterator:
            if str(cell) != patt:
                for val in list(patt):
                    cell[...] = str(cell).replace(val,'')

for row in range(BOARD_SIZE):
    patterns = get_common_patterns(possible[row,:])
    if len(patterns):
        print(F"row: {patterns}")
    remove_pattern_from_group(possible[row,:], patterns)
for col in range(BOARD_SIZE):
    patterns = get_common_patterns(possible[:,col])
    if len(patterns):
        print(F"col: {patterns}")
    remove_pattern_from_group(possible[:,col], patterns)
for row in range(0,BOARD_SIZE,3):
    for col in range(0,BOARD_SIZE,3):
        patterns = get_common_patterns(possible[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE])
        if len(patterns):
            print(F"sector: {patterns}")
        remove_pattern_from_group(possible[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE], patterns)

pretty_flat(possible.flatten())
pretty_flat(board.flatten())
clean_singletons(board,possible)

col: ['267']
sector: ['267']
+------------------------+------------------------+------------------------+
| 23468     38           |                  236   |          468      46   |
| 24568     58           | 25678    267     2567  | 46789    4689          |
|  3568    358           |  5678            3567  |  1678    168           |
+------------------------+------------------------+------------------------+
| 12579             25   |  267     267           |         12569    1679  |
| 25789    5789    258   |          267           | 24679   24569    4679  |
|  127                   |                        |  127      12           |
+------------------------+------------------------+------------------------+
|  1678             68   |  167             467   |  1468                  |
| 156789  15789    568   | 12567           24567  | 124689  124689   1469  |
| 13569    1359          |  1256            256   |  1269            169   |
+------------------------+---------------------

0

In [375]:
possible[5,0] = '7'
pretty_flat(possible.flatten())

+------------------------+------------------------+------------------------+
| 23468     38           |                  236   |          468      46   |
| 24568     58           | 25678    267     2567  | 46789    4689          |
|  3568    358           |  5678            3567  |  1678    168           |
+------------------------+------------------------+------------------------+
| 12579             25   |  267     267           |         12569    1679  |
| 25789    5789    258   |          267           | 24679   24569    4679  |
|   7                    |                        |  127      12           |
+------------------------+------------------------+------------------------+
|  1678             68   |  167             467   |  1468                  |
| 156789  15789    568   | 12567           24567  | 124689  124689   1469  |
| 13569    1359          |  1256            256   |  1269            169   |
+------------------------+------------------------+------------------------+